# Sistem Basis Data Perpustakaan - SQLite & Pandas

In [ ]:
import sqlite3
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

In [ ]:
cursor.execute("""
CREATE TABLE buku (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    judul TEXT,
    penulis TEXT,
    tahun INTEGER
);
""")

In [ ]:
cursor.execute("""
CREATE TABLE anggota (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nama TEXT,
    email TEXT
);
""")

In [ ]:
cursor.execute("""
CREATE TABLE peminjaman (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    id_anggota INTEGER,
    id_buku INTEGER,
    tgl_pinjam TEXT,
    tgl_kembali TEXT,
    FOREIGN KEY (id_anggota) REFERENCES anggota(id),
    FOREIGN KEY (id_buku) REFERENCES buku(id)
);
""")

In [ ]:
buku_data = [
    ("Pemrograman Python", "Andi Setiawan", 2020),
    ("Basis Data Lanjut", "Siti Aminah", 2019),
    ("Algoritma & Struktur Data", "Budi Santoso", 2021),
    ("Jaringan Komputer", "Rina Wulandari", 2018),
    ("Sistem Operasi", "Tono Pratama", 2022)
]
cursor.executemany("INSERT INTO buku (judul, penulis, tahun) VALUES (?, ?, ?)", buku_data)

anggota_data = [
    ("Dion Alif", "dion@gmail.com"),
    ("Siti Nur", "siti@gmail.com"),
    ("Rizki Mahendra", "rizki@gmail.com")
]
cursor.executemany("INSERT INTO anggota (nama, email) VALUES (?, ?)", anggota_data)
conn.commit()

In [ ]:
peminjaman_data = [
    (1, 1, "2025-01-10", "2025-01-15"),
    (1, 3, "2025-01-12", None),
    (2, 2, "2025-01-14", "2025-01-18"),
    (3, 4, "2025-01-16", "2025-01-22")
]
cursor.executemany("INSERT INTO peminjaman (id_anggota, id_buku, tgl_pinjam, tgl_kembali) VALUES (?, ?, ?, ?)", peminjaman_data)
conn.commit()

In [ ]:
import pandas as pd

df_peminjaman = pd.read_sql_query("""
SELECT peminjaman.id, anggota.nama, buku.judul, tgl_pinjam, tgl_kembali
FROM peminjaman
JOIN anggota ON peminjaman.id_anggota = anggota.id
JOIN buku ON peminjaman.id_buku = buku.id;
""", conn)
df_peminjaman

In [ ]:
df_peminjaman['tgl_pinjam'] = pd.to_datetime(df_peminjaman['tgl_pinjam'])
df_peminjaman['tgl_kembali'] = pd.to_datetime(df_peminjaman['tgl_kembali'])
df_peminjaman['durasi_hari'] = (df_peminjaman['tgl_kembali'] - df_peminjaman['tgl_pinjam']).dt.days
df_peminjaman

In [ ]:
import matplotlib.pyplot as plt

df_peminjaman['durasi_hari'].dropna().plot(kind='hist')
plt.title("Distribusi Durasi Peminjaman")
plt.xlabel("Hari")
plt.ylabel("Frekuensi")
plt.show()

df_group = df_peminjaman.groupby("nama").size()
df_group.plot(kind='bar')
plt.title("Jumlah Buku Dipinjam per Anggota")
plt.xlabel("Nama Anggota")
plt.ylabel("Jumlah Buku")
plt.show()